# vectorize the data 

In [1]:
from __future__ import print_function
 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

# Get the data 

In [4]:
batch_size = 64 
epochs = 1000  
latent_dim = 1024
num_samples = 10000 
data = "C:/Users/MSI/Desktop/Kookmin/Second semester/Deep Learning/assignment/translation/kor.txt"


input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data, encoding='UTF8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _, = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
 
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 3648
Number of unique input tokens: 74
Number of unique output tokens: 911
Max sequence length for inputs: 537
Max sequence length for outputs: 298


In [5]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
 
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
 
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
 
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
 

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
 model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
          epochs=epochs, validation_split=0.2)

Epoch 1/1000
46/46 [==============================] - 33s 596ms/step - loss: 0.2614 - val_loss: 0.4068
Epoch 2/1000
46/46 [==============================] - 25s 548ms/step - loss: 0.2258 - val_loss: 0.4013
Epoch 3/1000
46/46 [==============================] - 25s 543ms/step - loss: 0.2230 - val_loss: 0.3999
Epoch 4/1000
46/46 [==============================] - 25s 547ms/step - loss: 0.2229 - val_loss: 0.3995
Epoch 5/1000
46/46 [==============================] - 25s 554ms/step - loss: 0.2226 - val_loss: 0.3995
Epoch 6/1000
46/46 [==============================] - 25s 549ms/step - loss: 0.2228 - val_loss: 0.3978
Epoch 7/1000
46/46 [==============================] - 26s 557ms/step - loss: 0.2208 - val_loss: 0.3944
Epoch 8/1000
46/46 [==============================] - 25s 551ms/step - loss: 0.2165 - val_loss: 0.3974
Epoch 9/1000
46/46 [==============================] - 25s 554ms/step - loss: 0.2231 - val_loss: 0.3956
Epoch 10/1000
46/46 [==============================] - 25s 547ms/step - l

In [7]:
# Save model
model.save('C:/Users/MSI/Desktop/Kookmin/Second semester/Deep Learning/assignment/translation/try2.h5')

In [8]:

encoder_model = Model(encoder_inputs, encoder_states)
 
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
 
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [9]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
 
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
 
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
 
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
 
    return decoded_sentence

In [12]:
for seq_index in range(50):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: 가.

-
Input sentence: Hi.
Decoded sentence: 안녕!

-
Input sentence: Run!
Decoded sentence: 뛰어!

-
Input sentence: Run.
Decoded sentence: 뛰어!

-
Input sentence: Who?
Decoded sentence: 누구?

-
Input sentence: Wow!
Decoded sentence: 우와!

-
Input sentence: Fire!
Decoded sentence: 뛰어!

-
Input sentence: Help!
Decoded sentence: 도와줘!

-
Input sentence: Jump!
Decoded sentence: 빨리 오세요?

-
Input sentence: Jump.
Decoded sentence: 빨리 오세요?

-
Input sentence: Wait!
Decoded sentence: 기다려!

-
Input sentence: Wait!
Decoded sentence: 기다려!

-
Input sentence: Wait.
Decoded sentence: 기다려!

-
Input sentence: Begin.
Decoded sentence: 일어나!

-
Input sentence: Hello!
Decoded sentence: 안녕!

-
Input sentence: I see.
Decoded sentence: 나 정나 놨니다.

-
Input sentence: I try.
Decoded sentence: 나는 쿨하게 보이려고 노력했어.

-
Input sentence: I won!
Decoded sentence: 내가 이겼어!

-
Input sentence: Oh no!
Decoded sentence: 아니 이런!

-
Input sentence: Relax.
Decoded sentence: 여기해.

-
Input sentence: Sho